<a href="https://colab.research.google.com/github/aashrithresearch/pytorch_pathology/blob/main/lymphoma/lymphomapptx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install python-pptx -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 12.2 MB/s eta 0:00:00


In [3]:
from pptx import Presentation
from pptx.util import Inches
from datetime import datetime
from skimage import color
import glob
import cv2
import numpy as np
from io import BytesIO
from tqdm.autonotebook import tqdm
import PIL.Image as Image

<ipython-input-3-5f111744811c>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
title = "lymphoma classification reproduced"
date = datetime.today()
author = "aashrith yedavalli"
comments = "data and code taken from blog andrewjanowczyk.com "
pptxfname = "lymphomapp.pptx"

In [10]:
import glob

mask_files = glob.glob("/content/drive/MyDrive/lymphoma.tar/**/*.tif")

In [6]:
prs = Presentation()
prs.slide_width = Inches(10)
prs.slide_height = Inches(10)

blank_slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(blank_slide_layout)

slide.placeholders[0].text = title

tf = slide.placeholders[1].text_frame
tf.text = f'Date: {date}\n'
tf.text += f"Author: {author}\n"
tf.text += f"Comments: {comments}\n"

In [7]:
# Create a new slide with the desired layout
blank_slide_layout = prs.slide_layouts[6]
new_slide = prs.slides.add_slide(blank_slide_layout)

# Add text boxes for your sentences
left = Inches(1)  # Adjust position as needed
top = Inches(1)
width = Inches(8)
height = Inches(1)

# Add text box 1:
txBox1 = new_slide.shapes.add_textbox(left, top, width, height)
tf1 = txBox1.text_frame
tf1.text = "This is my lymphoma  project that I have created. The dataset contains 3 lymphoma classes, CLL, FL, and MCL. "

# Add text box 2:
top += Inches(1.5)  # Adjust vertical spacing
txBox2 = new_slide.shapes.add_textbox(left, top, width, height)
tf2 = txBox2.text_frame
tf2.text = "Here is the approach that I followed: first I converted the image to hdf5 file format. Then, I trained the dataset using the hdf5 file using unet github. Then, I visualized the results using matplotlib and tensorboardX. Lastly, I was able to produce outputs for each image and respective classes."

top += Inches(1.5)  # Adjust vertical spacing
txBox3 = new_slide.shapes.add_textbox(left, top, width, height)
tf3 = txBox3.text_frame
tf3.text = "After 100 epochs of training, the following metrics were: train loss = 0.0444, test loss = 6.6718, train acc = 0.9853, test acc = 0.4200. Using the tensorboardX, i was able to visualize all of the different metrics."

top += Inches(1.5)  # Adjust vertical spacing
txBox4 = new_slide.shapes.add_textbox(left, top, width, height)
tf4 = txBox4.text_frame
tf4.text = "What I did differently: I combined many of the steps together. For example, instead of having a PS_scikitlearn library to install, I just wrote the code for it in the same notebook, combining many steps together. Also, I added comments to a lot of places in the code so that I could understand different processes taking place. Furthermore, I had to change many of the filenames or libraries that were outdated, and had to debug a lot of code, in this process. Lastly and most importantly, during the training process, I was able to add a function that was able to calculate train and test accuracy, helping me notice how the model was overfitting on the dataset. In the future, I would add more augmentations, dropout, and regularization techniques to minimize overfitting."

In [9]:
def addimagetoslide(slide,img,left,top, height, width, resize = .5):
    res = cv2.resize(img , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC) #resizing
    image_stream = BytesIO()
    Image.fromarray(res).save(image_stream,format="PNG")

    pic = slide.shapes.add_picture(image_stream, left, top ,height,width)
    image_stream.close()

In [11]:
def blend2Images(img, mask):
    if (img.ndim == 3):
        img = color.rgb2gray(img)
    if (mask.ndim == 3):
        mask = color.rgb2gray(mask)
    img = img[:, :, None] * 1.0
    mask = mask[:, :, None] * 1.0
    out = np.concatenate((mask, img, mask), 2) * 255
    return out.astype('uint8')

In [12]:
import os

for mask_fname in tqdm(mask_files):
    blank_slide_layout = prs.slide_layouts[0]  # Assuming this layout is for content
    slide = prs.slides.add_slide(blank_slide_layout)

    orig_fname = mask_fname.replace("./masks", "./imgs").replace("_mask.png", ".tif")
    output_fname = mask_fname.replace("./masks", "./output").replace("_output.png", "_class.png")

    # Load and add images
    img = cv2.cvtColor(cv2.imread(orig_fname), cv2.COLOR_BGR2RGB)
    addimagetoslide(slide, img, Inches(0), Inches(0), Inches(5), Inches(5))

    mask = cv2.cvtColor(cv2.imread(mask_fname), cv2.COLOR_BGR2RGB)
    addimagetoslide(slide, mask, Inches(5), Inches(0), Inches(5), Inches(5))

    # Check if output file exists
    if os.path.exists(output_fname):
        output = cv2.cvtColor(cv2.imread(output_fname), cv2.COLOR_BGR2RGB)
        addimagetoslide(slide, img, Inches(0), Inches(0), Inches(4), Inches(4))
        addimagetoslide(slide, mask, Inches(4.5), Inches(0), Inches(4), Inches(4))
        addimagetoslide(slide, output, Inches(4.5), Inches(4.5), Inches(4), Inches(4))
        addimagetoslide(slide, blend2Images(output, mask), Inches(0), Inches(4.5), Inches(4), Inches(4))
        txBox = slide.shapes.add_textbox(Inches(9), Inches(0), Inches(7), Inches(8))
        # Add text box for metrics
        txBox = slide.shapes.add_textbox(Inches(10), Inches(0), Inches(6), Inches(4))  # Increased width
        tf = txBox.text_frame

        # Add metrics with smaller font size and handling division by zero
        tf.text = f"{orig_fname}\n"

        # Overall Pixel Agreement:
        tf.text += f"Overall Pixel Agreement: {(output==mask).mean():.4f}\n"
        # True Positive Rate:
        tp = (mask[output > 0] > 0).sum()
        fp = (output > 0).sum()
        if fp != 0:
            tf.text += f"True Positive Rate: {tp / fp:.4f}\n"
        else:
            tf.text += f"True Positive Rate: N/A (No positive predictions)\n"

        # False Positive Rate:
        fp = (mask[output == 0] > 0).sum()
        tn = (output == 0).sum()
        if tn != 0:
            tf.text += f"False Positive Rate: {fp / tn:.4f}\n"
        else:
            tf.text += f"False Positive Rate: N/A (No negative predictions)\n"

        # True Negative Rate:
        tn = (mask[output == 0] == 0).sum()
        fp = (output == 0).sum()
        if fp != 0:
            tf.text += f"True Negative Rate: {tn / fp:.4f}\n"
        else:
            tf.text += f"True Negative Rate: N/A (No negative predictions)\n"
        # False Negative Rate:
        fn = (mask[output > 0] == 0).sum()
        tp = (output > 0).sum()
        if tp != 0:
            tf.text += f"False Negative Rate: {fn / tp:.4f}\n"
        else:
            tf.text += f"False Negative Rate: N/A (No positive predictions)\n"
    else:
        print(f"Warning: Output file not found: {output_fname}")

# Save the presentation
prs.save(pptxfname)

  0%|          | 0/374 [00:00<?, ?it/s]